In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# Import Dataset noticas emol + latercera
df1 = pd.read_csv('chile_2021-01.csv')
df1.dropna(subset=['text'], inplace=True)
df4 = pd.read_csv('chile_2021-04.csv')
df4.dropna(subset=['text'], inplace=True)
df_emol = pd.concat([df1, df4], axis = 0)
df_emol.dropna(subset=['text'], inplace=True)

# Import Dataset noticas 24hrs + Biobio
df9 = pd.read_csv('chile_2020-09.csv')
df9.dropna(subset=['text'], inplace=True)
df10 = pd.read_csv('chile_2020-10.csv')
df10.dropna(subset=['text'], inplace=True)
df11 = pd.read_csv('chile_2020-11.csv')
df11.dropna(subset=['text'], inplace=True)
df12 = pd.read_csv('chile_2020-12.csv')
df12.dropna(subset=['text'], inplace=True)
df_24bio = pd.concat([df9, df10, df11, df12], axis = 0)
df_24bio.head(10)

df = pd.concat([df_emol, df_24bio], axis = 0)
df.dropna(subset=['text'], inplace=True)
print(df.shape)
df.head(10)

(62619, 9)


,id,country,media_outlet,url,title,text,date,year,id_journalist
0,3816243,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/ley-...,Ley de Migración: diputada Núñez (RN) formula ...,La diputada y vicepresidenta de Renovación Nac...,2021-01-01,2021.0,NaN
1,3816387,chile,elmostrador,https://www.elmostrador.cl/destacado/2021/01/0...,2021 arranca con 3.588 nuevos contagios y 52 f...,"Según el reporte de hoy, el Ministerio de Salu...",2021-01-01,2021.0,NaN
2,3816512,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/829-...,829 personas fueron detenidas durante noche de...,Carabineros reportó que hasta las 06:00 am. de...,2021-01-01,2021.0,NaN
3,3816630,chile,elmostrador,https://www.elmostrador.cl/tv/entrevistas/2021...,Diputado RN Tomás Fuentes: “Sichel le quita es...,Eventual pacto electoral con el Partido Republ...,2021-01-01,2021.0,NaN
4,3860172,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/323110-ca...,Desde alcaldes a Presidente: El intenso calend...,Un inusual e intenso calendario electoral se v...,2021-01-01,2021.0,NaN
5,3860359,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/323068-la...,Año Nuevo: Denuncian lanzamiento de fuegos art...,Apenas llegado el 2021 y a pesar de su prohibi...,2021-01-01,2021.0,NaN
6,3862819,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/323108-de...,Había más de 100 personas: Detienen a administ...,La tarde de este viernes se detuvo a un hombre...,2021-01-01,2021.0,NaN
7,3862940,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/323107-ni...,Niño de tres años falleció tras caer en una pi...,Un niño de tres años falleció este viernes lue...,2021-01-01,2021.0,NaN
8,3863068,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/323106-re...,Principalmente mujeres: Casi 1.5 millones de p...,"Hoy, 1 de enero de 2021, se inicia la segunda ...",2021-01-01,2021.0,NaN
9,3863220,chile,ahoranoticiasmega,https://www.meganoticias.cl/nacional/323105-in...,Incendio consumió al menos cuatro casas en la ...,La tarde de este viernes se desarrolló un ince...,2021-01-01,2021.0,NaN


## Número de noticieros

In [3]:
from pandasql import sqldf

q="""SELECT media_outlet FROM df GROUP BY media_outlet"""
noticieros=sqldf(q)
noticieros


,media_outlet
0,ahoranoticiasmega
1,biobiochile
2,elciudadano
3,elmostrador
4,emol
5,horas24
6,latercera


In [4]:
#emol y biobiochile # SPLIT 4
#Noticieros
#https://www.emol.com/noticias/Internacional/2021/01/01/1008227/senado-anula-veto-trump-defensa.html 
#https://www.biobiochile.cl/noticias/economia/negocios-y-empresas/2021/12/13/banco-de-espana-destaca-expansion-del-santander-y-bbva-en-america-latina.shtml


#Meganoticas, ciudadano, latercera, 24hrs # SPLIT 3
#https://www.meganoticias.cl/deportes/ 
#https://www.elciudadano.com/reportaje-investigacion/argentina-estallido-social-deudas-dejado-impunidad/12/13/  #3
#https://www.latercera.com/politica/noticia/siches-y-eventual-reunion-entre-bachelet-y-boric-entiendo-que-ella-esta-disponible-y-que-ha-venido-a-apoyar-y-votar-porque-no-da-lo-mismo-quien-gobierne/WHPZUOOG7ZFCHFWGLNOFZBBQIU/
#https://www.24horas.cl/politica/opiniones-divididas-genera-la-llegada-a-chile-de-la-expresidenta-michelle-bachelet-5099856


#--- inconcluso
#https://www.elmostrador.cl/dia/2021/12/13/suspendida-la-segunda-cumbre-de-la-runasur-promovida-por-evo-morales-en-peru/ #
#https://www.elmostrador.cl/noticias/opinion/2021/12/03/el-ser-humano-y-la-hembra-genero-y-politica-de-ayer-y-hoy/ #3
#---


In [5]:
#ELCIUDADANO, HORAS24, LATERCERA, MEGANOTICIAS -> SPLIT 3 

q="""SELECT * FROM df WHERE media_outlet='horas24' OR media_outlet='latercera' OR media_outlet='ahoranoticiasmega' OR media_outlet='elciudadano'"""
data1 = sqldf(q) #Split 3
data1['temas'] = data1.apply (lambda row: row['url'].split('/')[3], axis=1)

q="""SELECT temas FROM data1 GROUP BY temas"""
topics = sqldf(q)
print(topics[0:30])
print(topics[31:len(topics)])

                  temas
0            actualidad
1                animal
2                 artes
3       calidad-de-vida
4     cartas-ciudadanas
5                 chile
6    ciencia-tecnologia
7   ciudadanos-al-poder
8              columnas
9      conversacioneslt
10          coronavirus
11                culto
12                 data
13             deportes
14     derechos-humanos
15         eclipse-2020
16             economia
17            editorial
18            educacion
19         el-deportivo
20           especiales
21               genero
22        internacional
23             justicia
24   la-tercera-domingo
25        la-tercera-pm
26        la-tercera-tv
27        latinoamerica
28              mapuche
29              masdeco
                      temas
31                   mexico
32                  mineria
33                    mouse
34                    mundo
35                 nacional
36                 negocios
37              noticiasbbc
38                  opinion
39  

In [7]:
#EMOL Y BIOBIOCHILE, SPLIT 4
q2="""SELECT * FROM df WHERE media_outlet='emol' OR media_outlet='biobiochile'"""
data2 = sqldf(q2) #Split 4
data2['temas'] = data2.apply (lambda row: row['url'].split('/')[4], axis=1)

q2="""SELECT temas FROM data2 GROUP BY temas"""
topics2 = sqldf(q2)
print(topics2)


            temas
0           Autos
1        Deportes
2        Economia
3    Espectaculos
4   Internacional
5        Nacional
6      Tecnologia
7      Tendencias
8        deportes
9        economia
10  internacional
11       nacional
12       sociedad


## Temas en las urls de las noticias

{0: 'Catástrofes y Accidentes', 1: 'Ciencias y Tecnologia', 2: 'Crimen, delitos y justicia', 3: 'Cultura y Artes', 4: 'Deporte', 5: 'Ecologia', 6: 'Economia', 7: 'Mundo', 8: 'Politica y Conflictos', 9: 'Salud'}

In [10]:
#TEMAS EN LA URL DE LAS NOTICAS
dfx = pd.concat([data1, data2], axis = 0)

q="""SELECT temas FROM dfx GROUP BY temas"""
topics = sqldf(q)

print(topics[0:30])
print(topics[31:len(topics)])

                  temas
0                 Autos
1              Deportes
2              Economia
3          Espectaculos
4         Internacional
5              Nacional
6            Tecnologia
7            Tendencias
8            actualidad
9                animal
10                artes
11      calidad-de-vida
12    cartas-ciudadanas
13                chile
14   ciencia-tecnologia
15  ciudadanos-al-poder
16             columnas
17     conversacioneslt
18          coronavirus
19                culto
20                 data
21             deportes
22     derechos-humanos
23         eclipse-2020
24             economia
25            editorial
26            educacion
27         el-deportivo
28           especiales
29               genero
                      temas
31                 justicia
32       la-tercera-domingo
33            la-tercera-pm
34            la-tercera-tv
35            latinoamerica
36                  mapuche
37                  masdeco
38           medio-ambiente
39  

In [15]:
dfx.dropna(subset=['text'], inplace=True)
dfx['text'] = dfx.text.astype(str)
dfx.dtypes

id                 int64
country           object
media_outlet      object
url               object
title             object
text              object
date              object
year             float64
id_journalist     object
temas             object
target_topic      object
dtype: object

In [16]:
def topic_url(tema, url):
    temas_castastrofes = ['']
    temas_tecnologia = ['Tecnologia', 'ciencia-tecnologia', 'Autos']
    temas_crimen = ['justicia']
    temas_cultura = ['artes', 'Espectaculos', 'podcast', 'pueblos', 'turismo']
    temas_deporte = ['Deportes','deportes']
    temas_ecologia = ['medio-ambiente', 'animal', 'turismo', 'eclipse-2020']
    temas_economia = ['Economia', 'economia']
    temas_mundo = ['Internacional','mundo']
    temas_politica = ['politica']
    temas_salud = ['salud', 'coronavirus']
    
    words_accidente = ['accidente', 'catastrofe']
    words_crimen = ['muerte', 'crimen', 'delito', 'homicidio']
     
    for word in words_crimen:
        if(word in url):
            return "Crimen, delitos y justicia"
    for word in words_accidente:
        if(word in url):
            return "Catástrofes y Accidentes"
    if tema in temas_ecologia:
        return "Ecologia"    
    if tema in temas_economia:
        return "Economia"      
    if tema in temas_politica:
        return "Politica y Conflictos"
    if tema in temas_mundo:
        return "Mundo"
    if tema in temas_tecnologia:
        return "Ciencias y Tecnologia"
    if tema in  temas_salud:
        return "Salud"
    if tema in temas_deporte:
        return "Deporte"
    if tema in temas_cultura:
        return "Cultura y Artes"
    else:
        return "No definido"

In [17]:
testdf=dfx
print(testdf.shape)
testdf['target_topic'] = testdf.apply (lambda row: topic_url(row['temas'], row['url']), axis=1)

(57651, 11)


In [19]:
q="""SELECT * FROM testdf WHERE target_topic != 'No definido'"""
dftotal=sqldf(q)
dftotal.target_topic.value_counts()

Deporte                       6681
Mundo                         5806
Politica y Conflictos         3225
Economia                      2151
Crimen, delitos y justicia    1559
Ciencias y Tecnologia          699
Salud                          355
Catástrofes y Accidentes       312
Cultura y Artes                305
Ecologia                       159
Name: target_topic, dtype: int64

### Table to CSV

In [26]:
#dftotal.to_csv('datatotal.csv')
dftotal.to_csv('../../Tarea2/datatotal.csv')
dftotal.to_csv('../../Tarea3/datatotal.csv')